In [1]:
import pickle
from docuverse.utils import open_stream
from pymilvus import (
    MilvusClient,
    DataType,
    connections,
    utility
)

def load_cache(cache_file):
    return pickle.load(open_stream(cache_file))

# cache_file="/home/raduf/.local/share/elastic_ingestion/benchmark__beir_dev__quora____en__corpus.small.jsonl_512_100_True_all_gte-small.pickle.xz"
# cache_dense="/home/raduf/.local/share/elastic_ingestion/benchmark__beir_dev__quora__en__corpus.small.jsonl_512_100_True_all_slate.30m.english.rtrvr.pickle.xz"
# cache_hybrid="/home/raduf/.local/share/elastic_ingestion/benchmark__beir_dev__quora__en__corpus.small.jsonl_512_100_True_all_slate.30m.english.rtrvr.pickle.xz"
# ddata=load_cache(cache_dense)
# hdata=load_cache(cache_hybrid)
# data=ddata
# import json
# questions = [
#     "How can I get free gems in Clash of Clans?",
#     "How can I get free gems Clash of Clans?",
#     "How do you feel when someone upvotes your answer on Quora?",
#     "What are the best thriller movie in Hollywood?",
#     "What should someone do to overcome anxiety?"
# ]

In [4]:
True

True

In [14]:
# MODEL = ("/home/raduf/sandbox2/docuverse/models/slate.30m.english.rtrvr")
# MODEL = ("ibm-granite/granite-embedding-30m-english")
MODEL = ("ibm-granite/granite-embedding-english-r2")
data_list = []
from docuverse.utils.embeddings.dense_embedding_function import DenseEmbeddingFunction
model = DenseEmbeddingFunction(MODEL)

Running on the gpus:['NVIDIA GeForce RTX 4090', 'NVIDIA GeForce RTX 5090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090'], attention: sdpa
=== done initializing model


In [6]:
ddata = [
    {"text": "Apply these configuration details to nodes to ensure that you have a valid configuration. Host bus adapters and node canisters Each of the FlashSystem 7200 node canisters feature the optional adapters that are shown in this table. Table 1. Host interface adapters Supported number of adapters Ports Protocol Possible slots 0-2 4 32 Gb Fibre Channel 1,2 0-2 4 16 Gb Fibre Channel 1,2 0-2 2 25 Gb Ethernet (iWARP) 1,2 0-2 2 25 Gb Ethernet (RoCE) 1,2 0-1 in slot 1 only 2 (4-port adapter but only 2 ports are active.) 12 Gb SAS Expansion 1 FlashSystem 7200, IBM\u00ae Storage FlashSystem 9100, and FlashSystem 9200 can be connected over 16 or 32 Gbps Fibre Channel or 25 Gbps Ethernet. The 32 Gbps Fibre Channel adapter supports simultaneous SCSI and NVMeFC connections on the same port. The SAS expansion adapter is required to use 2076-24G or 4664-24G or 2076-92G or 4664-92G expansion enclosures. The 25 Gb adapters support iSCSI host attachment. FlashSystem 7200 node canisters also contain two USB ports and the on-board Ethernet ports shown in this table. Table 2. On-board Ethernet ports On-board Ethernet port Speed Functions 1 10 GbE Management IP, Service IP, Host I/O 2 10 GbE Secondary Management IP, Host I/O 3 10 GbE Host I/O 4 10 GbE Host I/O 1 GbE Technician Port - DHCP/DNS for direct attach service management Fibre Channel over Ethernet (FCoE) is not supported. For information about supported hardware, see the following website: www.ibm.com/support . Volumes Each volume is accessible from the two nodes in an I/O group. Each host network port can recognize up to eight paths to each logical unit (LU) that is presented by the system. The hosts must run a multipathing device driver before the multiple paths can resolve to a single device. "},
    {"text": "HyperSwap configuration by using interswitch links\nNote: ISLs must not be shared between private and public virtual fabrics. To implement private and public SANs with dedicated switches, any combination of supported switches can be used. For the list of supported switches and for supported switch partitioning and virtual fabric options, see the interoperability website: www.ibm.com/support Like for every managed disk, all control enclosures need access to the quorum disk by using the same storage system ports. If a storage system with active/passive controllers (such as IBM DS3000, IBM DS4000\u00ae, IBM DS5000, or IBM FAStT) is attached to a fabric, the storage system must be connected with both internal controllers to this fabric. By using FCIP, passive WDM, or active WDM for quorum site connectivity, you can add to the extension. The connections must be reliable. It is strictly required that the links from both production sites to the quorum site are independent and do not share any long-distance equipment. FCIP links are supported also for ISLs between the two production sites in public and private SANs. A private SAN and a public SAN can be routed across the same FCIP link. However, to ensure bandwidth to the private SAN, it is typically necessary to configure FCIP tunnels. Similarly, it is permissible to multiplex multiple ISL links across a DWDM link. Note: It is not required to UPS-protect FCIP routers or active WDM devices that are used only for the control enclosure-to-quorum communication. A HyperSwap configuration is supported only when the storage system that hosts the quorum disks supports extended quorum. Although the system can use other types of storage systems for providing quorum disks, access to these quorum disks is always through a single path. Additional bandwidth requirements A bandwidth equal to the peak write bandwidth (as sum from all hosts) is required for intersite communication between I/O groups. This bandwidth must be available in the private SAN. Additionally, you need intersite bandwidth in the public SAN for host-to-node communication if a host accesses nodes in the other sites. "}
]

In [7]:
def create_data(raw_data, embedding_name='qembedding'):
    embeddings = model.encode([d['text'] for d in raw_data], show_progress_bar=True)
    keys_to_keep = {"text"}
    data_list = [{**{k:v for k, v in d.items() if k in keys_to_keep}, '_id': d['id'], embedding_name:embeddings[i]} for i,d in enumerate(raw_data)]
    return data_list
data_list = create_data(ddata)
 #hdate_list = create_data(hdata)


NameError: name 'ddata' is not defined

In [9]:
ddata[0]['id']

NameError: name 'ddata' is not defined

In [ ]:
import time
from docuverse.utils.timer import timer


def test_search(vectors, vector_for_query=None, metric="IP", reingest=False, milvus_server_addr="test.db",
                use_connections=False, ingest_batch_size=-1, collection_name="test3", vector_field_name="qembedding"):
    truncate_dim = 384
    if ingest_batch_size < 0:
        ingest_batch_size = len(vectors)

    if vector_for_query is None:
        entities = vectors
        test = vectors[0:3]
    else:
        entities = vectors
        if isinstance(vector_for_query, list) and isinstance(vector_for_query[0], dict):
            test = [d[vector_field_name] for d in vector_for_query]# [{vector_field_name: e} for e in vector_for_query]
        else:
            test = vector_for_query

    if use_connections:
        client = connections
        init, host, port = milvus_server_addr.split(":")
        host = host.replace("//", "")
        client.connect(host=host, port=port)
        client1 = MilvusClient(milvus_server_addr)
    else:
        client = MilvusClient(milvus_server_addr)
        client1 = client


    if reingest or not client.has_collection(collection_name=collection_name):
        schema = client1.create_schema(auto_id=True, enable_dynamic_field=True, primary_field="id")

        schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
        schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=50000)
        schema.add_field(field_name="_id", datatype=DataType.VARCHAR, max_length=50000)
        schema.add_field(field_name=vector_field_name, datatype=DataType.FLOAT_VECTOR, dim=truncate_dim)

        index_params = client1.prepare_index_params()
        index_params.add_index(
            field_name=vector_field_name,
            index_type="FLAT",
            metric_type=metric,
            index_name=vector_field_name,
            params={"nlist": 1024}
        )

        client1.drop_collection(collection_name=collection_name)
        client.create_collection(
            collection_name=collection_name, schema=schema, index_params=index_params
        )
        for i in range(0, len(vectors), ingest_batch_size):
            client.insert(collection_name=collection_name, data=entities[i:i + ingest_batch_size])
        # insert_result = client.insert(collection_name=collection_name, data=entities)
        # print({k: v for k, v in insert_result.items() if k != 'ids'})
        client.load_collection(collection_name=collection_name)
        ingested_items = 0
        connections.connect(host="localhost", port=19530)
        utility.wait_for_index_building_complete(collection_name=collection_name, index_name=vector_field_name)
        # print(
        #     client.count_entities(collection_name=collection_name)
        # )
        tm = timer()
        start = time.time()
        while False and ingested_items < len(vectors)-1:
            res = client.get_collection_stats(collection_name=collection_name)
            ingested_items = res["row_count"]
            print(f"{tm.time_since_beginning()}: Currently ingested items: {ingested_items}")
            time.sleep(10)
        print(f"Ingested in {tm.time_since_beginning()} seconds.")
        print(client.list_indexes(collection_name=collection_name))
        print(client.describe_index(collection_name=collection_name, index_name=vector_field_name))

    return client.search(
        collection_name=collection_name,
        # data=[t[vector_field_name] for t in test],
        data=test,
        #data=test,
        search_params={"metric_type": metric, "params": {"nprobe": 100, "efSearch": 128}},
        # anns_field=vector_field_name,
        limit=10,
        output_fields=["text", "_id"],
    )

def print_answer(q, res):
    print(f"Question: {q['text'] if isinstance(q, dict) else q}")
    if len(res) == 0:
        print("  ** No results found. **")
    else:
        for r in res:
            print({'id': r['entity']['_id'], 'answer': r['entity']['text'], 'score': r['distance']})
        print("\n")

def test_setup(milvus_server_addr="test.db", reingest=False, use_connections=False, collection_name="test3", vector_field_name="qembedding", metric="IP"):
    global questions, score
    print(f"Testing {milvus_server_addr}")
    questions = data_list
    answers = test_search(data_list, questions, reingest=reingest, milvus_server_addr=milvus_server_addr,
                          use_connections=use_connections, collection_name=collection_name, vector_field_name=vector_field_name,
                          metric=metric)
    score = 0
    total = len(questions)

    for q, res in zip(questions, answers):
        print(res)
        if isinstance(q, dict):
            if q['_id'] in [r['entity']['_id'] for r in res[0:3]]:
                score += 1
            else:
                if len(res) > 0:
                    print_answer(q, res)
        else:
            print_answer(q, res)
    print(f"Score: {score * 1.0 / total:.2f}")

In [ ]:
data_list[0].keys()

In [ ]:
online_milvus = "http://localhost:19530"
file_milvus = "../test.db"
use_connections = True
conns = {
    "quora_docuverse": ("beir_quora_small_milvus_dense_512_100_08292024", "embeddings", "IP"),
    "test": ("test3", "qembedding", "IP")
}
test = conns["quora_docuverse"]
keys_to_keep = {"text"}
data_list = create_data(data, test[1])
dense_collection_name = "beir_quora_small_milvus_dense_512_100_10062024"
hybrid_collection_name = "beir_quora_new_milvus_hybrid_512_100_10062024"

In [ ]:
len(data_list)

In [ ]:
test_setup(online_milvus, reingest=True, use_connections=False, collection_name=test[0], vector_field_name=test[1], metric=test[2])

In [ ]:
def extract_embeddings(data, vector_field_name):
    return [di[vector_field_name] for di in data[:2]]

def quick_search(collection_name, test_data, metric="IP"):
    global client
    metric=metric
    res=client.search(
        collection_name=collection_name,
        data=test_data,
        search_params={"metric_type": metric, "params": {"nprobe": 100, "efSearch": 128}},
        limit=10,
        output_fields=["text", "id"],
    )
    return res

In [ ]:
num_questions = 10
questions = [dt['text'] for dt in data[:num_questions]]
client = MilvusClient("http://localhost:19530")
test_data=extract_embeddings(data_list[:num_questions], vector_field_name="embeddings")
dres = quick_search(collection_name=dense_collection_name, test_data=test_data)
hres = quick_search(collection_name=hybrid_collection_name, test_data=test_data)

In [ ]:
dvec = client.query(collection_name=dense_collection_name, filter="id == '12017-0-62'")
hvec = client.query(collection_name=hybrid_collection_name, filter="id == '12017-0-62'")

In [ ]:
[(r['entity']['id'], r['distance']) for r in dres[0]]

In [ ]:
[(r['entity']['id'], r['distance']) for r in hres[0]]

In [ ]:
for q, res in zip(questions, answers):
    print(f"Question: {q['text']}")
    if len(res) == 0:
        print("  ** No results found. **")
    else:
        for r in res:
            print({'id': r['entity']['_id'], 'answer': r['entity']['text'], 'score': r['distance']})
        print("\n")

In [ ]:
questions[0]['text']

In [7]:
client = MilvusClient("http://localhost:19530")
debug_index_name="nq_milvus_dense_granite149m_512_100_20250623"

In [8]:
[c for c in client.list_collections()if c.find(debug_index_name)>=0]

['nq_milvus_dense_granite149m_512_100_20250623']

In [15]:
print(client)

In [9]:
client.get_collection_stats(debug_index_name)

{'row_count': 3170119}

In [10]:
client.list_indexes(debug_index_name)

['embeddings', '_id']

In [12]:
client.describe_collection(debug_index_name)

{'collection_name': 'nq_milvus_dense_granite149m_512_100_20250623',
 'auto_id': True,
 'num_shards': 1,
 'description': 'nq_milvus_dense_granite149m_512_100_20250623',
 'fields': [{'field_id': 100,
   'name': '_id',
   'description': 'ID',
   'type': <DataType.INT64: 5>,
   'params': {},
   'auto_id': True,
   'is_primary': True},
  {'field_id': 101,
   'name': 'id',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 1000}},
  {'field_id': 102,
   'name': 'title',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 10000}},
  {'field_id': 103,
   'name': 'text',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 21000}},
  {'field_id': 104,
   'name': 'embeddings',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 768}}],
 'functions': [],
 'aliases': [],
 'collection_id': 461155848649118595,
 'consistency_level': 3,
 'properties': {},
 'num_partitions'

In [9]:
from pymilvus import utility, connections
connections.connect(host="localhost", port='19530')

In [ ]:
utility.index_building_progress("test2", "qembedding")


In [13]:
client.list_collections()
# [c for c in client.list_collections() if c.find("small")>=0]

['_qmsum_milvus_dense_granite149m_1024_100_20250307',
 'unified_search_short23k_milvus_dense_granite149m_512_100_20250307',
 'unified_search_short23k_milvus_dense_granite30m_512_100_20250623',
 'longembed_needle_milvus_dense_granite149m_16384_100_20250307',
 'unified_search_short23k_milvus_dense_granite149m_128_100_20250623',
 'nq_milvus_dense_granite149m_512_100_20250623',
 'unified_search_short23k_milvus_dense_granite149m_1024_100_20250307',
 '_courtlistener_Plain_Text_milvus_dense_granite149m_1024_100_20250307',
 '_qasper_abstract_milvus_dense_granite149m_4096_100_20250307',
 'longembed_narrativeqa_milvus_dense_granite149m_1024_100_20250307',
 'beir_mldr_milvus_bm25_512000_0_20250512',
 '_qasper_title_milvus_dense_granite149m_4096_100_20250307',
 '_stackoverflow_milvus_dense_granite149m_2048_100_20250307',
 'beir_mldr_milvus_bm25_8192_0_20250513_trim8k',
 'unified_search_full_milvus_dense_granite149m_1024_100_20250307',
 'unified_search_short23k_milvus_dense_granite47m_1024_100_2025

In [21]:
txt="aclput Command Purpose Sets the access control information of a file. Syntax aclput [ -i inAclFile ] [ -R ] [ -t acl_type ] [ -v ]FileObject Description The aclput command sets the access control information of the file object that is specified by the FileObject parameter. The command reads standard input for the access control information, unless you specify the -i flag. Note: If you are reading from standard input your entries must match the expected format of the access control information or you get an error message. Use the Ctrl-D key sequence to complete the session. Access Control List Access Control Lists form the core of protection for file system objects. Each file system object is uniquely associated with one piece of data, called ACL that defines the access rights to the object. ACL might consist of multiple Access Control Entries (ACEs), each defining one particular set of access rights for a user. Typically, ACE consists of information such as identification (to whom this ACE applies) and access rights (allow-read, deny-write). ACE might also capture information such as inheritance flags and alarm and audit flags. The format and enforcement of ACL data is entirely dependent on the ACL type in which they are defined. AIX\u00ae provides for existence of multiple ACL types on the operating system. The list of ACLs supported by a file system instance depends on the physical file system implementation for that file system instance. Flags Table 1. Flags Item Description -i inAclFile Specifies the input file for access control information. If the access control information in the file that is specified by the InAclFile parameter is not correct, when you try to apply it to a file, an error message that is preceded by an asterisk is added to the input file. Note: The size of the ACL information depends on the ACL type. -R Apply ACL to this directory and its children file system objects recursively. -t ACL_type Specifies the ACL type of the ACL information that is displayed. If option is not provided, the actual ACL data in its original ACL type is displayed. The supported ACL types are ACLX and NFS4. -v Verbose option. This option displays many comment lines as part of the ACL data display. This comment might help in understanding the details of complex ACL types. Security Access Control This command should be a standard user program and have the trusted computing base attribute. Auditing Events If the auditing subsystem is properly configured and is enabled, the aclput command generates the following audit record or event every time the command is run: Event Information FILE_WriteXacl Modification to access controls. RBAC users Attention RBAC users: This command can perform privileged operations. Only privileged users can run privileged operations. For more information about authorizations and privileges, see Privileged Command Database in Security. For a list of privileges and the authorizations that are associated with this command, see the lssecattr command or the getcmdattr subcommand. Examples To set the access control information for the status file with information from standard input, enter: aclput status attributes: SUID and then press the Ctrl-D sequence to exit the session. To set the access control information for the status file with information stored in the acldefs file, enter: aclput -i acldefs status To set the access control information for the status file with the same information used for the plans file, enter: aclget plans | aclput status To set the access control information for the status file with an edited version of the access control information for the plans file, you must enter two commands. First, enter: aclget -o acl plans This stores the access control information for the plans file in the acl file. Edit the information in the acl file by using your favorite editor. Then, enter: aclput -i acl status This second command takes the access control information in the acl file and puts it on the status file. Files Table 2. Files Item Description /usr/bin/aclput Contains the aclput command. Parent topic: a Related reference aclget Command auditpr Command Related information chmod Command Securing the network"
txt="what does bert look like from sesame street"
d=model.encode([txt])[0]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
qemb=model.encode([txt])[0]
search_params = {"metric_type": "IP", "params": {"nprobe": 100, "efSearch": 128}}

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
res=client.search(collection_name=debug_index_name, data=[qemb], search_param=search_params, limit=3, output_fields=["text", "_id"])
res

data: [[{'_id': 461155848651984474, 'distance': 0.9217905402183533, 'entity': {'text': 'Bert (Sesame Street)\nBart looks exactly like Bert, and Ernie once mistook Bart for Bert; but Bart wears a suit and fedora and is an entrepreneur who is just like Ernie in personality ("Bart\'s the name and selling\'s my game!"). Bart also tells several jokes and laughs at his own jokes and has combed hair which shocks Ernie who expected Bart to be as grumpy as Bert. Bert\'s Aunt Matilda is never shown, but is often talked about, usually concerning Bert buying presents for her for numerous reasons.', '_id': 461155848651984474}}, {'_id': 461155848651984462, 'distance': 0.9123363494873047, 'entity': {'text': "Bert (Sesame Street)\nBert is a muppet character on the long-running children's television show, Sesame Street. Bert was originally performed by Frank Oz. Since 1997, Muppeteer Eric Jacobson has been phased in as Bert's primary performer, like Grover. Bert has also made cameo appearances within T

In [20]:
client.describe_index("test1", "qembedding")

2025-10-20 17:06:14,809 [ERROR][handler]: RPC error: [describe_index], <MilvusException: (code=100, message=collection not found[database=default][collection=test1])>, <Time:{'RPC start': '2025-10-20 17:06:14.807659', 'RPC error': '2025-10-20 17:06:14.809392'}> (decorators.py:140)


MilvusException: <MilvusException: (code=100, message=collection not found[database=default][collection=test1])>

In [ ]:
client.describe_collection("test3")

In [ ]:
client.describe_index("beir_quora_small_milvus_dense_512_100_08292024", "embeddings")

In [ ]:
client.flush()

In [ ]:
text="How does Quora look to a moderator?"
text="I was suddenly logged off Gmail. I can't remember my Gmail password and just realized the recovery email is no longer alive. What can I do?"
qemb=model.encode([text])[0]
search_params = {"metric_type": "IP", "params": {"nprobe": 100, "efSearch": 128}}

In [ ]:
res=client.search(collection_name="ibmsw_milvus_hybrid_512_100_10022024", data=[d], search_param=search_params, limit=3, output_fields=["text", "_id"])

In [ ]:
res[0]

In [4]:
client=MilvusClient("/tmp/bla.db")